In [45]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv("not_transformed.csv")

In [3]:
# Dropping unnecessary columns
data2 = data[:] # keeping a copy for later
data.drop(['Unnamed: 0','display_id','campaign_id','uuid','document_id','platform','geo_location','day', 'hour', 'category_id','topic_id','entity_id','source_id','publisher_id'], axis=1, inplace=True)

In [5]:
# Converting to dummies
data = pd.get_dummies(data)

# Splitting into train and test

# X = data.ix[:, data.columns != 'clicked']
# y = data['clicked']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X = data[:37003]
y = data[37003:]

X_train = X.ix[:, X.columns != 'clicked']
X_test = y.ix[:, y.columns != 'clicked']

y_train = X['clicked']
y_test = y['clicked']

print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(37003, 43)
(14668, 43)
(37003,)
(14668,)


In [8]:
#fit Randome Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=50, criterion='entropy')
model = rfc.fit(X_train,y_train)

In [20]:
prediction = model.predict_proba(X_test)

In [27]:
click_prob = prediction[:,1]

In [96]:
#calculate MAP@12 score
test_index_range = data2[37003:].groupby('display_id').count()['ad_id'].tolist()
start = 0
num = 0
map12 = 0
df_test = data2[37003:].reset_index()
for p in test_index_range:
    end = start+p
    y_pred = click_prob.tolist()
    temp_pred = y_pred[start:end]
    temp = pd.DataFrame(df_test.ix[start:end-1][['ad_id','clicked']]).copy().reset_index().drop('index',axis=1)
    df=pd.DataFrame(temp_pred,columns=['prob'])
    temp = pd.concat([temp,df],axis=1)
    temp2= temp.sort_values('prob',ascending=False).reset_index().drop('index',axis=1)
    index = temp2.ix[temp2['clicked']==1].index[0]
    map12 = map12+(1.0/(index+1))
    start = end
print map12/len(test_index_range)

0.545879638984
